In [ ]:
%matplotlib notebook

# Frozen Lake

Frozen Lake is one of the environments in gym, ideal to play around with value and policy iteration:

https://gym.openai.com/envs/FrozenLake-v0/

The following code is based on [Justin Francis](https://github.com/wagonhelm/Value-Iteration)' solution.

You might need to install gym

`pip3 install -U gym`

or 

`conda install -c conda-forge gym`

In [ ]:
import numpy as np
import gym
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output

In [ ]:
env = gym.make('FrozenLake8x8-v0')

In [ ]:
n_states = env.observation_space.n
n_actions = env.action_space.n

print("States =",n_states,"; Actions =",n_actions)

Implemented By Using Pseudo Code From: <br />
[Reinforcement Learning: An Introduction](http://www.incompleteideas.net/sutton/book/the-book-2nd.html) <br />
By: Richard Sutton and Andrew Barto

<img src="images/value.png">

In [ ]:
values = np.zeros(n_states)
gamma = 0.999

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1,projection='3d')
x = np.outer(np.linspace(1,8,8), np.ones(8))
y = x.copy().T
surf=None

while True:
    delta = 0
    for state in reversed(range(n_states)):
        v = values[state]
        values[state] = np.max([sum([p*(r + gamma*values[s_]) for p, s_, r, _ in env.env.P[state][a]]) for a in range(env.env.nA)])
        delta = max(delta,abs(v-values[state]))

    # Show the found values
    if surf:
        surf.remove()
    z = values.reshape(8,8)
    surf=ax.plot_surface(x, y, z, cmap='seismic', rstride=1, cstride=1, linewidth=0, alpha=0.7)
    fig.canvas.draw()
    
    # Stop if too small the change
    if delta < 1e-3:
        break
    

In [ ]:
history = []
bestAverage = []
state = env.reset()
test_episodes = 1000

for i in range(1,test_episodes):
    while True:
        action = np.argmax([sum([p*(r + gamma*values[s_]) for p, s_, r, _ in env.env.P[state][a]]) for a in range(env.env.nA)])
        state, reward, done, info = env.step(action)

        if done:
            clear_output(wait=True)
            env.render()

            history.append(reward)
            env.reset()           
            
            break
        
    if len(history)>=100 and np.mean(history[i-100:i]) >= 0.30:
        bestAverage.append(np.mean(history[i-100:i]))
        
print('Best Average Score Per 100 Episodes in {} Episodes: {:4.3f}'.format(test_episodes, np.max(bestAverage)))

Value Iteration assumes that the entire MDP is known but in most cases the MDP is unknown.  Let's try and solve FrozenLake-v0 by trying to estimate all p(s',r|s,a) and state values and by using e-greedy Q-Learning.

In [ ]:
env = gym.make('FrozenLake-v0')

In [ ]:
n_states = env.observation_space.n
n_actions = env.action_space.n


In [ ]:
env.render()

We can also think of our states like the following.

In [ ]:
states = np.zeros([n_states])
for i in range(n_states):
    states[i] = i
states.reshape(4,4)

There are 4 possible actions with their associated labels

[0] = Left <br />
[1] = Down <br />
[2] = Right <br />
[3] = Up

In [ ]:
table = np.zeros([n_states,n_states,n_actions,4])

Table represents the number of initial states (**s**), future states (**s'**), number of possible actions(**a**), and the last 4 indexes represent:

[0] - Reward<br />
[1] - How many times action is taken given state (**#a|s**)<br />
[2] - How many times state transitioned to **s'** given **a** (**#s'|a**)<br />
[3] - Probability of transitioning to **s'** from **s** given action **a** (**p(s'|s,a)**)<br />

Since the rewards for state transitions do not change we do not need to calculate **p(s',r|s,a**)



Let's make a function that can evaluate all possible state transitions for any given state.

In [ ]:
def possibleStates(s):
    p_states = []
    for i in range(n_states):
        if np.max(table[s,i,:,[3]]):
            p_states.append(i)
    return p_states

def actionProbs(s):
    for i in possibleStates(s):
        print('-----------------------------------------------------')
        for a in range(n_actions):
            print('P: State: [{}] -> State: [{}] via Action: [{}] is {:4.4f}'.format(s,i,a,table[s,i,a,3]))


Let's evaluate the probability of each state transition from state[0] by sampling all possible actions from the initial state 100,000 times.  By [law of large numbers](https://www.youtube.com/watch?v=Ws63I3F7Moc) we will converge to the state transition probabilities.  

In [ ]:
for i in range(n_actions):
    for _ in range(10000):
        state = env.reset()
        action = i
        state2, reward, done, info = env.step(action)

        table[state,state2,action,0] = reward
        table[state,:,action,1] += 1
        table[state,state2,action,2] += 1
        table[state,:,action,3] = table[state,:,action,2] / table[state,:,action,1]

And now we can display the probability of transitioning to any possible future state given any action.

In [ ]:
actionProbs(env.reset())

So if we wanted to transition to state 4, the best possible action is [0] (LEFT) because from state 0 we will have a 66% chance of staying in state 0 and a 33% chance of transitioning to state 4.  

One way to solve this environment we will have to sample it a large amount of times and keep track of the estimated value of each state.  We will create a value table of 16 for each possible states.  We update our state value using the TD Target also known as Tabular TD(0) for estimating V(pi) the optimal values.  We will use learning rate alpha to determine the rate of update we apply to V(s).  The value can be considered our expected return in a given state.  We will explore the environment by behaving completely randomly updating the values at the same time which will be used later for another policy for completeing the environment.  This is also known as off-policy learning since we are not adapting the policy we are using to explore the environment.  Since it's exploring randomly it sometimes does not converge nicely without a very large amount of episodes. 

In [ ]:
value = np.zeros([n_states])
table = np.zeros([n_states,n_states,n_actions,4])

In [ ]:
gamma = 0.999
alpha = 0.1
history = []
state = env.reset()
n_episodes = 500000
n_steps = 500
n_solved = 0
for i in range(n_episodes):
    state = env.reset()
    for _ in range(n_steps):
        action = env.action_space.sample()
        state2, reward, done, info = env.step(action)

        table[state,state2,action,0] = reward
        table[state,:,action,1] += 1
        table[state,state2,action,2] += 1
        table[state,:,action,3] = table[state,:,action,2] / table[state,:,action,1]
        
        value[state] += alpha*(reward + gamma*value[state2] - value[state])
        
        state = state2
        if done:
            if reward == 1:
                n_solved += 1
            break
              
print('Solved {} Times In {} Episodes with average reward of {}'.format(n_solved, i+1, n_solved/n_episodes))

In [ ]:
value2 = value.reshape(4,4)
for i in range(4):
    print(value2[i,:])

So the holes and and goal point do not have any values because they do not lead to any future states.  When trying to estimate the best action to take we will take the probabilities of of all actions that lead to the best state and subtract them by the action probabilities for the worst state.  This ensures that actions that have a probability of leading to a worse state are not considered.  Since the holes and and finish point do not have values we will assign their values to be the expected reward for transitioning to that state.

In [ ]:
def bestState(s):
    beststate = s
    for i in possibleStates(s):
        if value[i] + np.max(table[:,i,:,0]) > value[beststate]:
            beststate = i
    return beststate

def worstState(s):
    worststate = s
    for i in possibleStates(s):
        if value[i] + np.max(table[:,i,:,0]) < value[worststate]:
            worststate = i
    return worststate

def bestAction(s):
    a = np.argmax((table[s,bestState(s),:,[3]])-(table[s,worstState(s),:,[3]]))
    return a

In [ ]:
history = []
for i in range(1,1000):
    while True:
        action = bestAction(state)
        state, reward, done, info = env.step(action)

        if done:
            history.append(reward)
            env.reset()
            break

    # Open AI's Standards For Solving
    if len(history)>=100 and np.mean(history[i-100:i]) >= 0.78:
        print('Solved in {} attempts'.format(i))
        break


There we have it by only behaving completely random and updating our expected state values we are able to solve the environment.  Here is the graph of the estimated state values. [1,1] represents the start state and [4,4] the finish state.

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
x = np.outer(np.linspace(1, 4, 4), np.ones(4))
y = x.copy().T
z = value2
ax.plot_surface(x, y, z, cmap='seismic', rstride=1, cstride=1, linewidth=0, alpha=0.7)
plt.show()
plt.close()


Also here is some old code I used for solving this environment using e-greedy Q-Learning (Action Value), which is a hell of alot less complicated then my previous method.  This e-greedy method also incorporates some random action noise which decays over each episode.  This method is on-policy and solves much faster because it's improving the Q-Value based policy every time step.

<img src="images/qlearn.png">

In [ ]:
def e_greedy(eps, Q, state, episode):
    
    if np.random.rand() > eps:
        action = np.argmax(Q[state,:]+np.random.randn(1, n_actions)/(episode/4))
    else:
        action = env.action_space.sample()
        eps -= 10**-5

    return action, eps

In [ ]:
Q = np.zeros([env.observation_space.n, env.action_space.n])
rewardTracker = []

alpha = 0.8
gamma = 0.95
eps = 0.1
numTrainingEpisodes = 5000
numTrainingSteps = 300

for episode in range(1,numTrainingEpisodes+1):  

    G = 0
    state = env.reset()

    for step in range(1,numTrainingSteps):

        action, eps = e_greedy(eps, Q, state, episode)
        state2, reward, done, info = env.step(action)
        Q[state,action] += alpha * (reward + gamma * np.max(Q[state2]) - Q[state,action])
        state = state2
        G += reward

    rewardTracker.append(G)

    if episode % (numTrainingEpisodes*.10) == 0 and episode != 0:
        print('Alpha {}  Gamma {}  Epsilon {:04.3f}  Episode {} of {}'.format(alpha, gamma, eps, episode, numTrainingEpisodes))
        print("Average Total Return: {}".format(sum(rewardTracker)/episode))

    if (sum(rewardTracker[episode-100:episode])/100.0) > .78:
        print('-------------------------------------------------------')
        print('Solved after {} episodes with average return of {}'.format(episode-100, sum(rewardTracker[episode-100:episode])/100.0))
        break


Here's the graphs for all the action values.

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(221,projection='3d')
x = np.outer(np.linspace(1, 4, 4), np.ones(4))
y = x.copy().T

ax.plot_surface(x, y, Q[:,0].reshape(4,4), cmap='magma', rstride=1, cstride=1, linewidth=0, alpha=0.7)
plt.title('Action[0] Left')

ax = fig.add_subplot(222,projection='3d')
ax.plot_surface(x, y, Q[:,1].reshape(4,4), cmap='magma', rstride=1, cstride=1, linewidth=1, alpha=0.7)
plt.title('Action[1] Down')

ax = fig.add_subplot(223,projection='3d')
ax.plot_surface(x, y, Q[:,2].reshape(4,4), cmap='magma', rstride=1, cstride=1, linewidth=1, alpha=0.7)
plt.title('Action[2] Right')

ax = fig.add_subplot(224,projection='3d')
ax.plot_surface(x, y, Q[:,3].reshape(4,4), cmap='magma', rstride=1, cstride=1, linewidth=1, alpha=0.7)
plt.title('Action[3] Up')

plt.show()
plt.close()